In [8]:
import numpy as np
import seaborn as sn
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout,BatchNormalization ,Activation
from tensorflow.keras.models import Sequential
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam

In [3]:
train_datagen = ImageDataGenerator( rescale = 1./255,
                                    validation_split=0.2,
                                    rotation_range = 20,
                                    horizontal_flip = True,
                                    vertical_flip=True,
                                    fill_mode = 'nearest'
                                  )

val_datagen = ImageDataGenerator(rescale = 1./255, 
                                 validation_split=0.2,
                                )
test_datagen = ImageDataGenerator(rescale = 1./255)

In [4]:
input_shape = (100,100)
src = 'ckplus'


train_data = train_datagen.flow_from_directory(src+'/train',
                                                target_size = input_shape,
                                                class_mode = 'categorical',
                                                batch_size = 64,
                                                subset='training'
                                              )

val_data = val_datagen.flow_from_directory(src+'/train',
                                            target_size = input_shape,
                                            class_mode = 'categorical',
                                            batch_size = 64,
                                            subset='validation'
                                          )

test_data = test_datagen.flow_from_directory(src+'/test',
                                              target_size = input_shape,
                                              class_mode = 'categorical',
                                              batch_size = 64
                                            )

Found 1372 images belonging to 7 classes.
Found 340 images belonging to 7 classes.
Found 430 images belonging to 7 classes.


In [5]:
train_steps = train_data.n//train_data.batch_size
val_steps = val_data.n//val_data.batch_size
test_steps = test_data.n//test_data.batch_size

In [9]:
# VGG16 model
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(input_shape[0], input_shape[1], 3))

# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False
    
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  # Assuming 7 classes for emotions

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 3, 3, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 512)               2359808   
                                                                 
 batch_normalization (Batch  (None, 512)               2048      
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 512)               0         
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                      

In [10]:
# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
checkpoint = ModelCheckpoint('emotion_model.h5', save_best_only=True)

# Training the model
history = model.fit(train_data,
                    steps_per_epoch=train_steps,
                    epochs=100,
                    validation_data=val_data,
                    validation_steps=val_steps,
                    callbacks=[reduce_lr, checkpoint]
                   )

# Evaluate on test data
test_loss, test_accuracy = model.evaluate(test_data, steps=test_steps)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')



Epoch 1/100


21/21 [==============================] - 58s 3s/step - loss: 1.6721 - accuracy: 0.4037 - val_loss: 1.9522 - val_accuracy: 0.4594 - lr: 0.0010


c:\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/100
21/21 [==============================] - 61s 3s/step - loss: 1.1115 - accuracy: 0.6193 - val_loss: 1.8907 - val_accuracy: 0.4625 - lr: 0.0010
Epoch 3/100
21/21 [==============================] - 62s 3s/step - loss: 0.9262 - accuracy: 0.6789 - val_loss: 1.6379 - val_accuracy: 0.4781 - lr: 0.0010
Epoch 4/100
21/21 [==============================] - 66s 3s/step - loss: 0.7996 - accuracy: 0.7194 - val_loss: 1.4150 - val_accuracy: 0.5312 - lr: 0.0010
Epoch 5/100
21/21 [==============================] - 72s 3s/step - loss: 0.7330 - accuracy: 0.7431 - val_loss: 1.1933 - val_accuracy: 0.5813 - lr: 0.0010
Epoch 6/100
21/21 [==============================] - 74s 4s/step - loss: 0.6850 - accuracy: 0.7439 - val_loss: 1.0341 - val_accuracy: 0.6125 - lr: 0.0010
Epoch 7/100
21/21 [==============================] - 108s 5s/step - loss: 0.5977 - accuracy: 0.7872 - val_loss: 1.2141 - val_accuracy: 0.5375 - lr: 0.0010
Epoch 8/100
21/21 [==============================] - 120s 6s/step - loss: 0